<a href="https://colab.research.google.com/github/minhnbvnu/ssbc/blob/main/SSBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers datasets huggingface_hub evaluate tensorflow

## Chuẩn bị dữ liệu

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset

sickr_sts_dataset = load_dataset("mteb/sickr-sts")

In [ ]:
from datasets import DatasetDict

train_testvalidation = sickr_sts_dataset['test'].train_test_split(test_size=0.2)
test_validation = train_testvalidation['test'].train_test_split(test_size=0.5)


sickr_sts_dataset = DatasetDict({
    'train': train_testvalidation['train'],
    'test': test_validation['test'],
    'validation': test_validation['train'],
})

In [ ]:
sickr_sts_dataset["test"][0]

{'sentence1': 'A large group of cheerleaders is walking in a parade',
 'sentence2': 'The cheerleaders are not parading and are not wearing black, pink and white uniforms',
 'score': 3.5}

In [ ]:
def process_data_label(dataset):
  labels = []
  for record in dataset:
    labels.append(int(record["score"]))
  return labels

train_label_column = process_data_label(sickr_sts_dataset["train"])
test_label_column = process_data_label(sickr_sts_dataset["test"])
validation_label_column = process_data_label(sickr_sts_dataset["validation"])


sickr_sts_dataset["train"] = sickr_sts_dataset["train"].add_column("label",train_label_column)
sickr_sts_dataset["test"] = sickr_sts_dataset["test"].add_column("label", test_label_column)
sickr_sts_dataset["validation"] = sickr_sts_dataset["validation"].add_column("label", validation_label_column)


Flattening the indices:   0%|          | 0/794 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/100 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/99 [00:00<?, ? examples/s]

In [ ]:
sickr_sts_dataset["test"][5]

{'sentence1': 'The man is hiking in the woods',
 'sentence2': 'The man is not hiking in the woods',
 'score': 4.4,
 'label': 4}

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Create a preprocessing function to tokenize `text` and truncate sequences to be no longer than DistilBERT's maximum input length:

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True)

In [ ]:
tokenized_sickr_sts = sickr_sts_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/794 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

In [ ]:
tokenized_sickr_sts["train"][0]

{'sentence1': 'The blonde girl with the pink top is smiling and wearing funny glasses with a large nose attached',
 'sentence2': 'The blonde girl with the pink top is happily smiling and wearing funny glasses with a large nose attached',
 'score': 4.8,
 'label': 4,
 'input_ids': [101,
  1996,
  9081,
  2611,
  2007,
  1996,
  5061,
  2327,
  2003,
  5629,
  1998,
  4147,
  6057,
  7877,
  2007,
  1037,
  2312,
  4451,
  4987,
  102,
  1996,
  9081,
  2611,
  2007,
  1996,
  5061,
  2327,
  2003,
  11361,
  5629,
  1998,
  4147,
  6057,
  7877,
  2007,
  1037,
  2312,
  4451,
  4987,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

## Đánh giá

In [ ]:
import evaluate

spearmanr = evaluate.load("spearmanr")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    print(predictions)
    preds = np.argmax(predictions, axis=1)
    print(preds)
    element_counts = {}

    for element in preds:
      if element in element_counts:
          element_counts[element] += 1
      else:
          element_counts[element] = 1

    for element, count in element_counts.items():
      print(f"{element}: {count} times")
    return spearmanr.compute(predictions=preds, references=labels)

## Huấn luyện

In [ ]:
id2label = {1: "NOT SIMILAR", 1: "SLIGHTLY SIMILAR", 2: "MODERATELY SIMILAR", 3: "SIMILAR", 4: "VERY SIMILAR", 5: "EXTREMELY SIMILAR"}
label2id = {val: key for key, val in id2label.items()}

In [ ]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=6, id2label=id2label, label2id=label2id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_set = model.prepare_tf_dataset(
    tokenized_sickr_sts["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_sickr_sts["validation"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
from transformers import create_optimizer

batch_size = 16
num_epochs = 5
batches_per_epoch = len(tokenized_sickr_sts["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

model.compile(optimizer=optimizer)

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [ ]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="ssbc_model_spearman_6_labels",
    tokenizer=tokenizer,
)

In [ ]:
callbacks = [metric_callback, push_to_hub_callback]

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=5, callbacks=callbacks)

Epoch 1/5
 5/49 [==>...........................] - ETA: 2:55 - loss: 1.5357

InvalidArgumentError: ignored

## Kết quả

In [ ]:
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
from transformers import pipeline
from datasets import load_dataset

model = TFAutoModelForSequenceClassification.from_pretrained("minhnb/ssbc_model_spearman_6_labels")

In [ ]:
for record in sickr_sts_dataset['test']:
  sentence1 = record["sentence1"]
  sentence2 = record["sentence2"]
  tokenizer = AutoTokenizer.from_pretrained("minhnb/ssbc_model_spearman_6_labels")
  inputs = tokenizer(sentence1, sentence2, return_tensors="tf")
  logits = model(**inputs).logits
  predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
  print(sentence1, sentence2, record["score"], predicted_class_id, id2label[predicted_class_id])